# NL2SQL Practice


이번 실습에서는:
- SQLite 메모리 DB를 사용해 SCOTT 샘플 데이터베이스 (테이블)를 로드합니다.
- 기본적인 SQL 쿼리를 실행하고 결과를 확인하는 방법을 연습합니다.
- NL2SQL을 실습해봅니다.


In [3]:
import sqlite3 # Built-in Python library for working with SQLite databases
import pandas as pd

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# create cursor object to execute SQL statements
cur = conn.cursor()


def q(sql, params=None):
    """
    Helper function to run a SQL query and return the result as a Pandas DataFrame.

    Args:
        sql (str): SQL query string .
        params (list/tuple, optional): Parameters for parameterized queries (to avoid SQL injection).

    Returns:
        pandas.DataFrame: Query results as a DataFrame.
    """

    df = pd.read_sql_query(sql, conn, params=params or [])
    return df


## The sample database (SCOTT)

테이블 정보

- **EMP**: 직원 정보 (empno, ename, job, mgr, hiredate, sal, comm, deptno)
- **DEPT**: 부서 정보 (deptno, dname, loc)
- **SALGRADE**: 급여 등급 구간 정보 (grade, losal, hisal)

다음과 같은 쿼리를 실행해 테이블 구조와 데이터를 직접 확인할 수 있습니다.
- `SELECT * FROM EMP LIMIT 5;`
- `SELECT * FROM DEPT;`


In [4]:
cur.executescript("""
DROP TABLE IF EXISTS EMP;
DROP TABLE IF EXISTS DEPT;
DROP TABLE IF EXISTS SALGRADE;
DROP TABLE IF EXISTS BONUS;

CREATE TABLE DEPT (
  DEPTNO INTEGER PRIMARY KEY,
  DNAME  TEXT,
  LOC    TEXT
);

CREATE TABLE EMP (
  EMPNO    INTEGER PRIMARY KEY,
  ENAME    TEXT,
  JOB      TEXT,
  MGR      INTEGER,
  HIREDATE TEXT,   -- SQLite는 DATE 타입을 엄격히 강제하지 않아 보통 TEXT로 저장
  SAL      INTEGER,
  COMM     INTEGER,
  DEPTNO   INTEGER,
  FOREIGN KEY (DEPTNO) REFERENCES DEPT(DEPTNO)
);

CREATE TABLE SALGRADE (
  GRADE INTEGER,
  LOSAL INTEGER,
  HISAL INTEGER
);

CREATE TABLE BONUS (
  ENAME TEXT,
  JOB   TEXT,
  SAL   INTEGER,
  COMM  INTEGER
);

INSERT INTO DEPT VALUES
(10,'ACCOUNTING','NEW YORK'),
(20,'RESEARCH','DALLAS'),
(30,'SALES','CHICAGO'),
(40,'OPERATIONS','BOSTON');

INSERT INTO EMP VALUES
(7369,'SMITH','CLERK',7902,'1980-12-17',800,NULL,20),
(7499,'ALLEN','SALESMAN',7698,'1981-02-20',1600,300,30),
(7521,'WARD','SALESMAN',7698,'1981-02-22',1250,500,30),
(7566,'JONES','MANAGER',7839,'1981-04-02',2975,NULL,20),
(7654,'MARTIN','SALESMAN',7698,'1981-09-28',1250,1400,30),
(7698,'BLAKE','MANAGER',7839,'1981-05-01',2850,NULL,30),
(7782,'CLARK','MANAGER',7839,'1981-06-09',2450,NULL,10),
(7788,'SCOTT','ANALYST',7566,'1987-07-13',3000,NULL,20),
(7839,'KING','PRESIDENT',NULL,'1981-11-17',5000,NULL,10),
(7844,'TURNER','SALESMAN',7698,'1981-09-08',1500,0,30),
(7876,'ADAMS','CLERK',7788,'1987-07-13',1100,NULL,20),
(7900,'JAMES','CLERK',7698,'1981-12-03',950,NULL,30),
(7902,'FORD','ANALYST',7566,'1981-12-03',3000,NULL,20),
(7934,'MILLER','CLERK',7782,'1982-01-23',1300,NULL,10);

INSERT INTO SALGRADE VALUES
(1,  700, 1200),
(2, 1201, 1400),
(3, 1401, 2000),
(4, 2001, 3000),
(5, 3001, 9999);
""")

# Commit
conn.commit()


# Quick sanity check: show the first 5 rows from EMP
q("SELECT * FROM EMP LIMIT 5;")


,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902,1980-12-17,800,NaN,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300.0,30
2,7521,WARD,SALESMAN,7698,1981-02-22,1250,500.0,30
3,7566,JONES,MANAGER,7839,1981-04-02,2975,NaN,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250,1400.0,30


## Part A. SQL Warm-up (run SQL directly)

- 먼저 SQL을 직접 작성하면서 기본 쿼리 실행을 연습합니다.
- 한번에 SQL 한문장씩 실행하고, 출력 결과를 확인해보세요.


In [10]:
# 월급이 2000이상인 직원들을 월급 기준 내림차순으로 정렬해서 출력

q("""
SELECT ename, job, sal
FROM emp
WHERE sal >= 2000
ORDER BY sal DESC;
""")


,ENAME,JOB,SAL
0,KING,PRESIDENT,5000
1,SCOTT,ANALYST,3000
2,FORD,ANALYST,3000
3,JONES,MANAGER,2975
4,BLAKE,MANAGER,2850
5,CLARK,MANAGER,2450


In [11]:
# 직원 이름/직무, 부서명/부서위치를 출력하되, 부서 번호 -> 이름 순으로 정렬

q("""
SELECT e.ename, e.job, d.dname, d.loc
FROM emp e
JOIN dept d ON e.deptno = d.deptno
ORDER BY d.deptno, e.ename;
""")


,ENAME,JOB,DNAME,LOC
0,CLARK,MANAGER,ACCOUNTING,NEW YORK
1,KING,PRESIDENT,ACCOUNTING,NEW YORK
2,MILLER,CLERK,ACCOUNTING,NEW YORK
3,ADAMS,CLERK,RESEARCH,DALLAS
4,FORD,ANALYST,RESEARCH,DALLAS
5,JONES,MANAGER,RESEARCH,DALLAS
6,SCOTT,ANALYST,RESEARCH,DALLAS
7,SMITH,CLERK,RESEARCH,DALLAS
8,ALLEN,SALESMAN,SALES,CHICAGO
9,BLAKE,MANAGER,SALES,CHICAGO


In [12]:
# 부서별 평균 월급을 계산해 평균 월급이 높은 부서부터 출력

q("""
SELECT d.dname, ROUND(AVG(e.sal), 1) AS avg_sal
FROM dept d
JOIN emp e ON e.deptno = d.deptno
GROUP BY d.dname
ORDER BY avg_sal DESC;
""")


,DNAME,avg_sal
0,ACCOUNTING,2916.7
1,RESEARCH,2175.0
2,SALES,1566.7


In [13]:
# 커미션이 null이면 0으로 처리해 월급 + 커미션 (총 급여)를 계산하고, 총 급여가 높은 순으로 출력

q("""
SELECT ename,
       sal + COALESCE(comm, 0) AS total_pay
FROM emp
ORDER BY total_pay DESC;
""")


,ENAME,total_pay
0,KING,5000
1,SCOTT,3000
2,FORD,3000
3,JONES,2975
4,BLAKE,2850
5,MARTIN,2650
6,CLARK,2450
7,ALLEN,1900
8,WARD,1750
9,TURNER,1500


## Part B. NL2SQL

- 해당 섹션에서는 사용자가 자연어 질문 (NLQ)을 입력하면, 모델이 이에 해당하는 SQL 쿼리를 생성합니다.
- 생성된 SQL은 이 노트북에서 만든 SQLite 데이터베이스에서 바로 실행됩니다.
- `os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"` 부분을 고쳐주세요

In [27]:
# Install dependencies
!pip -q install openai pydantic

import os
from getpass import getpass
from openai import OpenAI


# PASTE YOUR API KEY HERE
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE" #### FIXME

client = OpenAI()
MODEL = "gpt-4o-mini"


In [15]:
SCHEMA_CARD = """
You are generating SQL for a SQLite database with the classic SCOTT schema.

Tables:
- EMP(empno, ename, job, mgr, hiredate, sal, comm, deptno)
- DEPT(deptno, dname, loc)
- SALGRADE(grade, losal, hisal)
- BONUS(ename, job, sal, comm)  -- rarely used

Joins:
- EMP.deptno = DEPT.deptno
- Salary grade: EMP.sal BETWEEN SALGRADE.losal AND SALGRADE.hisal

SQLite notes:
- Use COALESCE(x, 0) for NULL handling.
- Use LIMIT n for top-n.
- Dates are stored as text like '1981-11-17'.
"""
print(SCHEMA_CARD)



You are generating SQL for a SQLite database with the classic SCOTT schema.

Tables:
- EMP(empno, ename, job, mgr, hiredate, sal, comm, deptno)
- DEPT(deptno, dname, loc)
- SALGRADE(grade, losal, hisal)
- BONUS(ename, job, sal, comm)  -- rarely used

Joins:
- EMP.deptno = DEPT.deptno
- Salary grade: EMP.sal BETWEEN SALGRADE.losal AND SALGRADE.hisal

SQLite notes:
- Use COALESCE(x, 0) for NULL handling.
- Use LIMIT n for top-n.
- Dates are stored as text like '1981-11-17'.



In [16]:
import re
from pydantic import BaseModel, Field


# Define the structured output format we want from the LLM
class NL2SQLResult(BaseModel):

    # The model must return exactly ONE SQLite SELECT (read-only) query
    sql: str = Field(description="A single SQLite SELECT query only. No INSERT/UPDATE/DELETE/DDL.")

    # Short explanation (1–2 lines) describing why the SQL answers the question
    explanation: str = Field(description="1-2 lines: why this SQL answers the question (Korean).")




In [17]:
# Keywords that we explicitly block to prevent any data modification or risky commands
BANNED = [
    "insert", "update", "delete", "drop", "alter", "create", "replace",
    "attach", "detach", "pragma", "vacuum", "truncate"
]

def sanitize_sql(sql: str) -> str:
    """Clean and validate the SQL returned by the LLM to ensure it is safe and well-formed."""
    s = sql.strip()

    # remove code fence
    s = re.sub(r"^```sql\s*", "", s, flags=re.IGNORECASE).strip()
    s = re.sub(r"^```\s*", "", s).strip()
    s = re.sub(r"\s*```$", "", s).strip()

    #
    parts = [p.strip() for p in s.split(";") if p.strip()]
    if len(parts) != 1:
        raise ValueError("Only ONE SQL statement is allowed.")
    s = parts[0] + ";"

    low = s.lower()
    if not (low.startswith("select") or low.startswith("with")):
        raise ValueError("Only SELECT/WITH queries are allowed.")

    for kw in BANNED:
        if re.search(rf"\b{kw}\b", low):
            raise ValueError(f"Disallowed keyword detected: {kw}")

    return s


In [18]:
def llm_nl2sql(nl_question: str) -> NL2SQLResult:
    """Ask the LLM to convert a natural-language question into a safe SQLite SELECT query."""

    instructions = (
        "You are an expert SQL engineer.\n"
        "Return ONLY one SQLite SELECT query that answers the user question.\n"
        "Use only the tables/columns provided.\n"
        "Do not modify data. No DDL/DML.\n"
        "Prefer simple queries. Use JOIN/GROUP BY/HAVING/subquery when needed.\n"
        "If the question is ambiguous, make a reasonable assumption and mention it briefly in explanation.\n"
    )

    # Call the OpenAI API with Structured Outputs so the response matches NL2SQLResult
    resp = client.responses.parse(
        model=MODEL,
        instructions=instructions,
        input=[
            {"role": "user", "content": SCHEMA_CARD},
            {"role": "user", "content": f"User question (Korean): {nl_question}"}
        ],
        text_format=NL2SQLResult,
    )

    # Extract the parsed structured output
    out = resp.output_parsed

    # Validate the generated SQL before executing it
    out.sql = sanitize_sql(out.sql)
    return out

In [24]:
def run_nl2sql(nl_question: str, retry: bool = True):
    # 1) Generate SQL from the natural-language question
    out = llm_nl2sql(nl_question)
    print("NLQ:", nl_question)
    print("\nSQL:\n", out.sql)
    print("\nDescription:", out.explanation)

    # 2) Execute the generated SQL on the SQLite dataset
    try:
        df = q(out.sql)
        return df

    except Exception as e:
        # If execution fails, print the SQLite error message
        print("\nError Message:", str(e))

        if not retry:
            return None

        # 3) One-time automatic fix: ask the LLM to correct the SQL using the error message
        fix_prompt = f"""
The SQL failed on SQLite. Fix it.
- Return a single SQLite SELECT query.
- Keep it minimal.
User question: {nl_question}
SQL you wrote:
{out.sql}
SQLite error:
{str(e)}
"""
        # Call the LLM again to produce a corrected SQL query
        resp = client.responses.parse(
            model=MODEL,
            instructions="Fix the SQL. Output only one SELECT query and 1-2 line explanation in Korean.",
            input=[
                {"role": "user", "content": SCHEMA_CARD},
                {"role": "user", "content": fix_prompt}
            ],
            text_format=NL2SQLResult,
        )
        fixed = resp.output_parsed
        fixed.sql = sanitize_sql(fixed.sql)

        print("\nRevised SQL:\n", fixed.sql)
        print("\nRevised Description:", fixed.explanation)

        try:
            return q(fixed.sql)
        except Exception as e2:
            print("\nStill Fails:", str(e2))
            return None


In [25]:
QUESTIONS = [

    # Part A의 SQL들
    "월급이 2000이상인 직원들을 월급 기준 내림차순으로 정렬해서 출력해줘",
    "직원 이름/직무, 부서명/부서위치를 출력하되, 부서 번호 -> 이름 순으로 정렬해줘",
    "부서별 평균 월급을 계산해 평균 월급이 높은 부서부터 출력해줘",
    "커미션이 null이면 0으로 처리해 월급 + 커미션 (총 급여)를 계산하고, 총 급여가 높은 순으로 출력해줘",

    # 랭킹
    "이 회사에서 월급(SAL)을 가장 많이 받는 TOP 3는 누구고, 각자 부서는 어디야?",
    "커미션(COMM)까지 합친 총 보상(sal+comm)이 가장 큰 사람 TOP 5를 보여줘.",
    "월급이 평균보다 높은 사람들만 이름과 월급을 보여줘. (회사 전체 평균 기준)",

    # 조직/관계(SELF-JOIN)
    "각 직원의 '상사 이름'을 같이 보여줘. (상사가 없는 사람도 포함)",
    "상사(MGR)별로 부하직원 수를 세어서, 부하가 많은 상사부터 정렬해줘.",

    # 부서 비교(GROUP BY)
    "부서별로 직원 수와 평균 월급을 같이 보여줘. 평균 월급이 높은 순으로!",
    "부서별로 (최고 월급 - 최저 월급) 격차가 큰 부서부터 보여줘.",

    # 조건/NULL
    "커미션이 있는 사람만 보여줘. 그리고 커미션이 월급보다 큰 사람도 찾아줘.",
    "입사연도(hiredate의 연도)별로 몇 명이 입사했는지 보여줘.",

    # 조인 2개 이상(급여등급)
    "급여 등급(SALGRADE)별로 직원 수를 보여줘. 등급 낮은 순으로.",
    "급여 등급이 4 이상인 사람들의 이름, 월급, 등급을 보여줘.",

    # HAVING(그룹 조건)
    "직업(JOB)별 평균 월급을 보여주되, 직원 수가 2명 이상인 직업만 보여줘.",
]

for i, qtext in enumerate(QUESTIONS, 1):
    print(f"{i:02d}. {qtext}")


01. 월급이 2000이상인 직원들을 월급 기준 내림차순으로 정렬해서 출력해줘
02. 직원 이름/직무, 부서명/부서위치를 출력하되, 부서 번호 -> 이름 순으로 정렬해줘
03. 부서별 평균 월급을 계산해 평균 월급이 높은 부서부터 출력해줘
04. 커미션이 null이면 0으로 처리해 월급 + 커미션 (총 급여)를 계산하고, 총 급여가 높은 순으로 출력해줘
05. 이 회사에서 월급(SAL)을 가장 많이 받는 TOP 3는 누구고, 각자 부서는 어디야?
06. 커미션(COMM)까지 합친 총 보상(sal+comm)이 가장 큰 사람 TOP 5를 보여줘.
07. 월급이 평균보다 높은 사람들만 이름과 월급을 보여줘. (회사 전체 평균 기준)
08. 각 직원의 '상사 이름'을 같이 보여줘. (상사가 없는 사람도 포함)
09. 상사(MGR)별로 부하직원 수를 세어서, 부하가 많은 상사부터 정렬해줘.
10. 부서별로 직원 수와 평균 월급을 같이 보여줘. 평균 월급이 높은 순으로!
11. 부서별로 (최고 월급 - 최저 월급) 격차가 큰 부서부터 보여줘.
12. 커미션이 있는 사람만 보여줘. 그리고 커미션이 월급보다 큰 사람도 찾아줘.
13. 입사연도(hiredate의 연도)별로 몇 명이 입사했는지 보여줘.
14. 급여 등급(SALGRADE)별로 직원 수를 보여줘. 등급 낮은 순으로.
15. 급여 등급이 4 이상인 사람들의 이름, 월급, 등급을 보여줘.
16. 직업(JOB)별 평균 월급을 보여주되, 직원 수가 2명 이상인 직업만 보여줘.


In [26]:
idx = int(input("풀어보고 싶은 질문 번호를 입력하세요 (1~{}): ".format(len(QUESTIONS))))
chosen = QUESTIONS[idx - 1]
df = run_nl2sql(chosen)
df


풀어보고 싶은 질문 번호를 입력하세요 (1~16): 1
NLQ: 월급이 2000이상인 직원들을 월급 기준 내림차순으로 정렬해서 출력해줘

SQL:
 SELECT empno, ename, sal FROM EMP WHERE sal >= 2000 ORDER BY sal DESC;

Description: 월급이 2000 이상인 직원들을 조회하고, 월급 기준으로 내림차순으로 정렬하기 위해 sal 컬럼을 기준으로 정렬한 쿼리입니다.


,EMPNO,ENAME,SAL
0,7839,KING,5000
1,7788,SCOTT,3000
2,7902,FORD,3000
3,7566,JONES,2975
4,7698,BLAKE,2850
5,7782,CLARK,2450
